In [1]:
#following tutorial here https://medium.com/@kevaldekivadiya2415/fine-tuning-llama-2-your-path-to-chemistry-text-perfection-aa4c54ff5790
from dotenv import load_dotenv
import os
load_dotenv()


True

In [2]:
# Load Medical Meadow QA dataset
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("medalpaca/medical_meadow_medqa", split="train")

print(f"Dataset Size: {len(dataset)}")
print(dataset[randrange(len(dataset))])


# Dataset Size: 10178

Found cached dataset json (/Users/juliev/.cache/huggingface/datasets/medalpaca___json/medalpaca--medical_meadow_medqa-18cf88a1c277db4e/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


Dataset Size: 10178
{'instruction': 'Please answer with one of the option in the bracket', 'input': "Q:A 53-year-old woman presents with a feeling of pelvic pressure which worsens with prolonged standing, pain on sexual intercourse, and lower back pain. She reports no urinary or fecal incontinence. She is G3P3 with no history of any gynecological disease and is premenopausal. All pregnancies were without complication and resolved with full-term vaginal deliveries. The patient has sex with her husband who is her single sexual partner and uses oral contraceptives. Her vital signs are within normal limits and physical examination is unremarkable. A gynecological examination reveals bulging of the posterior vaginal wall in the lower portion of the vagina which increases in the upright position and Valsalva maneuver. The cervix is in its normal position. The uterus is not enlarged, ovaries are nonpalpable. Damage to which of the following structures might contribute to the patient’s conditi

In [3]:
print(dataset[0]['input'])

print(dataset[0]['output'])

Q:A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?? 
{'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': 'Ciprofloxacin', 'D': 'Doxycycline', 'E': 'Nitrofurantoin'},
E: Nitrofurantoin


In [4]:
def format_prompt(sample):
    return f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{sample["instruction"]}

### Input:
{sample["input"]}

### Response:
{sample["output"]}
"""

In [5]:
from random import randrange

print(format_prompt(dataset[randrange(len(dataset))]))


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please answer with one of the option in the bracket

### Input:
Q:A 23-year-old woman presents to the emergency department after being found unresponsive by her friends. The patient is an IV drug user and her friends came over and found her passed out in her room. The patient presented to the emergency department 2 days ago after being involved in a bar fight where she broke her nose and had it treated and packed with gauze. Her temperature is 99.3°F (37.4°C), blood pressure is 90/48 mmHg, pulse is 150/min, respirations are 24/min, and oxygen saturation is 97% on room air. Physical exam is notable for an obtunded woman with nasal packing and EKG tags from her last hospital stay, as well as a purpuric rash on her arms and legs. Her arms have track marks on them and blisters. Which of the following is the best nex

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model name
model_name = "meta-llama/Llama-2-7b-chat-hf"
use_flash_attention = False
token = os.environ.get("HUGGINGFACE_TOKEN")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
    torch_dtype=torch.float16,
    token = token)

model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

AttributeError: module 'torch.distributed' has no attribute 'is_initialized'

In [3]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
)
# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

/Users/juliev/miniforge3/envs/metrics/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


AttributeError: module 'lib' has no attribute 'X509_V_FLAG_NOTIFY_POLICY'

In [4]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="finetuned-llama-7b-chat-hf-med",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False
)

RuntimeError: Failed to import transformers.training_args because of the following error (look up to see its traceback):
module 'lib' has no attribute 'X509_V_FLAG_NOTIFY_POLICY'

In [ ]:
# Train
trainer.train()

# Save model
trainer.save_model()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

# Load finetuned LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)

In [ ]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("medalpaca/medical_meadow_medqa", split="train")
sample = dataset[randrange(len(dataset))]

prompt = f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{sample["instruction"]}

### Input:
{sample["input"]}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = model.generate(input_ids=input_ids, max_new_tokens=512, do_sample=True, top_p=0.6,temperature=0.9)

print(f"Instruction:\n{sample['instruction']}\n")
print(f"Input:\n{sample['input']}\n")
print(f"Generated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}\n")
print(f"Ground Truth:\n{sample['output']}")